In [1]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, year, month, dayofmonth, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType
import pandas as pd 
import numpy as np

In [3]:
import sys
import os
project_root = os.path.abspath('..')
print(f"root:{project_root}")
if project_root not in sys.path:
    sys.path.append(project_root)

root:c:\Users\romul\Repositories\Airbnb_Data_Warehouse


In [4]:
fact_chuvas = pd.read_csv(f"{project_root}/data/raw/dados_pluviometricos_consolidados.csv")

C:\Users\romul\AppData\Local\Temp\ipykernel_10612\3938555641.py:1: DtypeWarning: Columns (3,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  fact_chuvas = pd.read_csv(f"{project_root}/data/raw/dados_pluviometricos_consolidados.csv")


In [5]:
listings = pd.read_csv(f"{project_root}/data/raw/listings.csv")

In [6]:
listings[listings['host_is_superhost']=='t']

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
1,25026,https://www.airbnb.com/rooms/25026,20250624193519,2025-07-04,city scrape,Beautiful Modern Decorated Studio in Copacabana,"**Fully renovated in Dec 2022, new kitchen, n...",Copacabana is a lively neighborhood and the ap...,https://a0.muscache.com/pictures/7c08fa4f-1d7b...,102840,...,4.93,4.85,4.65,NaN,f,1,1,0,0,1.71
2,35764,https://www.airbnb.com/rooms/35764,20250624193519,2025-06-25,city scrape,COPACABANA SEA BREEZE - RIO - 25 X Superhost,Our newly renovated studio is located in the b...,Our guests will experience living with a local...,https://a0.muscache.com/pictures/23782972/1d3e...,153691,...,4.95,4.95,4.89,NaN,f,1,1,0,0,2.88
3,48305,https://www.airbnb.com/rooms/48305,20250624193519,2025-06-26,city scrape,Bright 6bed Penthouse Seconds from Beach,Enter Bossa Nova's history by staying in the v...,Enter Bossa Nova history by staying in the ver...,https://a0.muscache.com/pictures/miso/Hosting-...,70933,...,4.84,4.95,4.59,NaN,t,6,5,1,0,1.05
5,49179,https://www.airbnb.com/rooms/49179,20250624193519,2025-06-27,city scrape,Djalma Ocean View Studio with Amazing Views,Beautiful 12th Floor Ocean View Studio steps f...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,224192,...,4.93,4.91,4.75,NaN,t,11,11,0,0,1.19
7,51703,https://www.airbnb.com/rooms/51703,20250624193519,2025-06-27,city scrape,"Ocean view, block of the beach, Copacabana,Ipa...",Oh....i love to host and to meet people intere...,"It is located in the hotspot of Rio, wonderful...",https://a0.muscache.com/pictures/d0295650-2b1e...,238091,...,4.92,4.93,4.67,NaN,f,2,2,0,0,1.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42553,1449836422986444435,https://www.airbnb.com/rooms/1449836422986444435,20250624193519,2025-06-28,city scrape,Mini apartamento em Copacabana,This room is an independent space in a well lo...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,27581763,...,NaN,NaN,NaN,NaN,t,3,3,0,0,NaN
42560,1450014170108315634,https://www.airbnb.com/rooms/1450014170108315634,20250624193519,2025-06-25,city scrape,Private room in Arporador,You'll have a great time at this comfortable p...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,45156129,...,NaN,NaN,NaN,NaN,f,4,1,3,0,NaN
42565,1450082923922623983,https://www.airbnb.com/rooms/1450082923922623983,20250624193519,2025-06-26,city scrape,Cama em Dorm Misto (9) c/Varanda,"Single bed in room with nine beds, located on ...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,37776540,...,NaN,NaN,NaN,NaN,t,8,0,4,4,NaN
42566,1450108079751966655,https://www.airbnb.com/rooms/1450108079751966655,20250624193519,2025-06-27,city scrape,Apart 1 Quarto Rua Sá Ferreira Posto 6 Copa/ M...,"Apartment posto 6 of Copacabana, cozy atmosphe...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,4232062,...,NaN,NaN,NaN,NaN,t,8,8,0,0,NaN


In [7]:
len(listings['id'])

42572

In [8]:
#callendar = pd.read_csv(f"{project_root}/data/raw/calendar.csv")

In [9]:
reviews = pd.read_csv(f"{project_root}/data/raw/reviews.csv")

In [10]:
len(reviews.listing_id.unique())

33949

## Criando notas artificias para as reviews.

In [11]:
media_n = 0.6
desvio_n = 0.2
nota_max  =5
nota_min =0
def distribuition_generator(elements =len(reviews), media_n = 0.6,desvio_n = 0.2,nota_max  =5 ,nota_min =0):
    distribuicao = np.random.normal(media_n,desvio_n,elements)
    distribuicao = np.round(distribuicao*nota_max,decimals=1)
    distribuicao = np.clip(distribuicao,nota_min,nota_max)
    return pd.Series(distribuicao.flatten())

In [12]:
notas_artificiais = {
    'nota_comunicacao': [0.4,0.3],
    'nota_limpeza' : [0.6,0.2],
    'nota_custo_beneficio' : [0.6,0.2]
}
for dist in notas_artificiais:
    media = notas_artificiais[dist][0]
    desvio = notas_artificiais[dist][1]
    reviews[dist] = distribuition_generator(media_n= media,desvio_n= desvio)    

In [13]:
reviews.reset_index(drop=True)

,listing_id,id,date,reviewer_id,reviewer_name,comments,nota_comunicacao,nota_limpeza,nota_custo_beneficio
0,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...,4.5,1.5,2.4
1,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...,0.1,3.6,1.5
2,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...,0.0,2.9,2.4
3,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r<b...,0.8,4.5,3.7
4,17878,147594,2010-12-01,219338,David,The apt was nice and in a great location only ...,2.0,4.8,1.9
...,...,...,...,...,...,...,...,...,...
1069272,1447261809655430116,1449312583557631186,2025-06-22,502578145,Lylyan,"Eu e meu namorados adoramos a estadia, a local...",3.0,2.9,0.9
1069273,1447342549635319128,1450044909045473576,2025-06-23,523455314,Alice,Ambiente super acolhedor. O André e a Andressa...,1.3,4.7,4.7
1069274,1448141046834354537,1449348675550278871,2025-06-22,261457678,Thiago,"Ap muito lindo, ficamos apaixonada tudo organi...",1.4,3.0,2.9
1069275,1448141046834354537,1452327802375005883,2025-06-26,703349982,Ramon,O ap é lindo. Amamos cada detalhe. Tudo estava...,1.7,3.3,3.4


## Retirando colunas redundantes csv das chuvas.

In [14]:
fact_chuvas= fact_chuvas.fillna(0)

In [15]:
fact_chuvas.describe()

,05min,10min,15min,01h,04h,24h,96h
count,1.325946e+07,1.325946e+07,1.325946e+07,1.325946e+07,1.325946e+07,1.325946e+07,1.325946e+07
mean,1.343947e-05,2.687893e-05,4.031840e-05,1.612736e-04,6.447776e-04,3.760003e-03,1.449253e-02
std,6.194792e-03,1.187741e-02,1.721065e-02,5.363399e-02,1.354062e-01,4.851202e-01,1.222633e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,7.800000e+00,1.440000e+01,2.100000e+01,4.660000e+01,6.180000e+01,1.136000e+02,1.402000e+02


In [ ]:
fact_chuvas['15 min'].dtype

0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
           ... 
13259451    0.0
13259452    0.0
13259453    0.0
13259454    0.0
13259455    0.0
Name: 15 min, Length: 13259456, dtype: object

In [ ]:
fact_chuvas[fact_chuvas['15 min']=='ND']# ND aparentemente são valores Não digitalizados.

,localizacao,Dia,Hora,HBV,15 min,01 h,04 h,24 h,96 h,05 min,10 min,05min,10min,15min,01h,04h,24h,96h
8533,alto_da_boa_vista,30/03/2015,20:15:00,ND,ND,ND,ND,ND,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8537,alto_da_boa_vista,30/03/2015,21:15:00,0.0,ND,0.0,15.6,63.8,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35971,alto_da_boa_vista,10/01/2016,16:45:00,ND,ND,ND,ND,ND,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35972,alto_da_boa_vista,10/01/2016,17:00:00,HBV,ND,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45814,alto_da_boa_vista,22/04/2016,04:30:00,ND,ND,ND,ND,ND,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13167737,vidigal,07/11/2024,14:35:00,0.0,ND,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13174716,vidigal,01/12/2024,20:15:00,0.0,ND,0.0,0.8,0.8,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13188616,vidigal,19/01/2025,02:40:00,0.0,ND,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13188876,vidigal,20/01/2025,00:25:00,0.0,ND,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#fact_chuvas['15 min'] = fact_chuvas['15 min'].astype(float)  + fact_chuvas['15min'].astype(float)
#fact_chuvas['01 h'] = fact_chuvas['01h']  + fact_chuvas['01 h'] 
#fact_chuvas['15 min'] = fact_chuvas['15 min']  + fact_chuvas['15min'] 
#fact_chuvas['15 min'] = fact_chuvas['15 min']  + fact_chuvas['15min'] 

ValueError: could not convert string to float: 'ND'

In [ ]:
fact_chuvas.describe()